In [37]:
import os, sys

import torch
import spacy
import numpy as np

nlp = spacy.load('en_core_web_sm')

In [38]:
os.chdir('/home/hugal/AIC/WIA/Ourseq2seq/')
import seq2seq

#Loading the dataset

In [39]:

test_src = open('/home/hugal/AIC/WIA/writingPrompts/test.wp_source', 'r')
test_tgt = open('/home/hugal/AIC/WIA/writingPrompts/test.wp_target', 'r')

tst_src = test_src.readlines()
tst_tgt = test_tgt.readlines()

test_src.close()
test_tgt.close()

In [40]:
print(len(tst_src))
print(len(tst_tgt))

15138
15138


In [41]:
for i in range(len(tst_src)):
    tst_src[i] = tst_src[i].split()[3:]
    tst_tgt[i] = tst_tgt[i].split()[3:]

In [42]:
def voc(data):
    dict = {}
    for l in data:
        for w in l:
            if w not in dict.keys():
                dict[w] = len(dict)
    return dict

def convert(data, vocab):
    d = []
    for l in data:
        c = []
        for w in l:
            c.append(vocab[w])
        c.reverse()
        d.append(c)
    return d

def padding(data_c, pad, ma):
    l = [len(e) for e in data_c]
    m = min(np.amax(l), ma)
    print(m)
    for i in range(len(l)):
        if l[i] > m:
            l[i] = m
            data_c[i] = data_c[i][:m]
        else:
            for j in range(m-l[i]):
                data_c[i].append(pad)
    z = list(zip(l, data_c))
    z.sort(reverse=True)
    l, data_c = zip(*z)
    return l, data_c
    

In [43]:
vocab = voc(tst_src + tst_tgt)

tst_src_c = convert(tst_src, vocab)
tst_tgt_c = convert(tst_tgt, vocab)

pad = '<pad>'
sos = '<sos>'
eos = '<eos>'
vocab[pad] = len(vocab)
vocab[sos] = len(vocab)
vocab[eos] = len(vocab)
tst_tgt_c[:].insert(0, vocab[sos])
tst_tgt_c[:].append(vocab[eos])
l_src, tst_src_p = padding(tst_src_c, vocab[pad], 1000)
l_tgt, tst_tgt_p = padding(tst_tgt_c, vocab[pad], 1000)
tst_src_t = torch.LongTensor(tst_src_p)
tst_tgt_t = torch.LongTensor(tst_tgt_p)

72


1000


In [44]:
print(tst_src_p[:5])
print(l_src[:5])

([22, 629, 333, 5, 287, 474, 83, 82, 688, 962, 85, 191, 390, 403, 312, 399, 1808, 22, 82, 141, 8, 838, 797, 895, 424, 59, 4029, 421, 2, 609, 6999, 3, 73, 170, 107, 311, 797, 403, 73, 240, 226, 403, 82, 312, 539, 103, 5282, 17, 8352, 358, 59, 4389, 3, 191, 2933, 333, 141, 17, 341, 93, 5, 2703, 37, 191, 459, 1206, 118, 107, 227, 226, 2670, 103], [111, 1163, 421, 3694, 494, 3087, 474, 59, 3038, 20, 3, 502, 425, 1283, 343, 22, 1441, 93, 550, 494, 3087, 474, 59, 3038, 865, 3, 502, 367, 155, 22, 4121, 48, 3552, 160, 1453, 5, 121, 357, 59, 72, 686, 2, 479, 22, 1630, 3, 341, 8, 59, 8296, 59, 672, 59, 4417, 59, 2655, 59, 1172, 59, 4243, 22, 72, 37, 2, 10025, 5, 3428, 1220, 320, 1195, 143094, 143094], [226, 1477, 100, 710, 411, 22, 4456, 5914, 5538, 2, 1865, 37, 19, 2199, 3, 5913, 396, 59, 967, 396, 123, 22, 3364, 5912, 2934, 1965, 3, 61, 5050, 10, 28, 2203, 23, 22, 2603, 37, 216, 609, 5911, 2132, 5910, 23, 22, 5909, 5908, 5907, 3, 17, 520, 5906, 37, 19, 4395, 650, 515, 5905, 3, 2888, 23, 22, 26

# Training

In [45]:
enc = seq2seq.Encoder(len(vocab), 20, 50, 1, 'cuda', vocab[pad])
dec = seq2seq.Decoder(len(vocab), 20, 50, 1, 'cuda', vocab[pad],vocab[sos], vocab[eos])

In [46]:
enc.to('cuda')
dec.to('cuda')
opt_enc = torch.optim.SparseAdam(enc.parameters())
opt_dec = torch.optim.SparseAdam(dec.parameters())

In [47]:
out, h_n = enc(tst_src_t[:4], torch.LongTensor(l_src[:4]))
output, h = dec(tst)

In [48]:
out

PackedSequence(data=tensor([[-0.1996, -0.0966,  0.1830,  ...,  0.1054, -0.0640, -0.1072],
        [ 0.1853,  0.0122, -0.2204,  ..., -0.1052,  0.0139,  0.0234],
        [-0.3172, -0.3463, -0.0004,  ..., -0.0717, -0.1594, -0.3158],
        ...,
        [ 0.0381,  0.0776,  0.1953,  ...,  0.2030, -0.1006, -0.0111],
        [ 0.1823,  0.0701, -0.0473,  ..., -0.0498, -0.1219, -0.2254],
        [ 0.0015, -0.0931,  0.1120,  ...,  0.1557, -0.2391, -0.2401]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>), batch_sizes=tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 1, 1],
       grad_fn=<PackPaddedBackward>))

#Saving weights

#Generating

#Saving generated examples